In [2]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_excel("./bin_data.xlsx")
df = pd.read_excel("./bin_data.xlsx")
df.head()

,年龄,收入,孩子数量,是否违约
0,46,0,0,0
1,34,3200,4,1
2,31,3300,3,1
3,39,1500,0,1
4,32,0,3,0


In [4]:
def catgory_bin(df,variable,dic1,dic2):
    """离散变量再分箱函数
    输入：
    1. df 为数据框
    2. variable_name 为待再分箱的变量名,以字符串形式给出
    3. dic1为新分箱的字典，key为新箱子的名字以字符串给出，value为原来的值，value均以原本值的类型给出
    4. dic2为新分箱的字典，key为新箱子的名字以字符串给出，value为筛选条件，value均以pd.query()的条件给出
    输出：
    重新分箱之后的数据框"""
    dic ={}
    for key1,value1 in dic1.items():
        index = df.loc[df[variable] == value1].index
        dic[key1] = index
    for key2,value2 in dic2.items():
        index = (df.query(expr = value2)).index
        dic[key2] = index
    for key,value in dic.items():
        df.loc[value,variable] = key
    print("变量%s分箱完成,分箱结果为%s" % (variable,df[variable].unique()))
    return df

In [5]:
df = catgory_bin(df,"孩子数量",dic1={"无子女":0,"一胎":1,"两胎":2},
                  dic2 = {"多胎":"孩子数量 >=3 & 孩子数量 < 5","非人类":"孩子数量 >=5"})
df.head()

变量孩子数量分箱完成,分箱结果为['无子女' '多胎' '两胎' '一胎' '非人类']


,年龄,收入,孩子数量,是否违约
0,46,0,无子女,0
1,34,3200,多胎,1
2,31,3300,多胎,1
3,39,1500,无子女,1
4,32,0,多胎,0


In [19]:
def customize_bin(df,variable,**kwargs):
    """自定义分箱的函数，可以针对离散变量和连续变量
    输入：
    1. df为待分箱的数据的数据框
    2. variable为待分箱的变量名
    3. **dic为关键字参数,给定形式为json格式。**json
    **{"dic1":{key为新箱子的名字以字符串给出，value为原来的值，value均以原本值的类型给出},
    "dic2":key为新箱子的名字以字符串给出，value为筛选条件，value均以pd.query()的条件给出}
    4. 如：**{"dic1":{"多胎":"孩子数量 >=3 & 孩子数量 < 5","非人类":"孩子数量 >=5"},"dic2":{"无子女":0,"一胎":1,"两胎":2}}
    输出：
    分箱完毕的数据框"""
    conditions = len(kwargs.keys()) 
    if conditions ==1: # 判断是连续变量
        dic1 = kwargs.get("dic1")
        dic = {}
        for key1,value1 in dic1.items():
            index = (df.query(expr = value1)).index
            dic[key1] = index
    elif conditions ==2: # 判断是离散变量
        dic1 = kwargs.get("dic1")
        dic2 = kwargs.get("dic2")
        dic = {}
        for key1,value1 in dic1.items():
            index = (df.query(expr = value1)).index
            dic[key1] = index
        for key2,value2 in dic2.items():
            index = df.loc[df[variable] == value2].index
            dic[key2] = index
        for key,value in dic.items():
            df.loc[value,variable] = key
        print("变量<%s>分箱完成,分箱结果为%s" % (variable,df[variable].unique()))
        return df
    else:
        print("请输入正确的关键字参数中key个数")

    import itertools
    value_list = list(itertools.chain(*dic.values())) 
    if len(value_list) != len(df):  #判断分箱的样本行数是否等于数据框样本数
        print("待分箱的数据与原始数据长度不符，请检查待分箱数据数量")
    else:
        for key,value in dic.items():
            df.loc[value,variable] = key
        print("变量<%s>分箱完成,分箱结果为%s" % (variable,df[variable].unique()))
        return df

In [20]:
df11 = customize_bin(df=data,variable="孩子数量",**{"dic1":{"多胎":"孩子数量 >=3 & 孩子数量 < 5","非人类":"孩子数量 >=5"},
                                           "dic2":{"无子女":0,"一胎":1,"两胎":2}})
df11.head()

变量<孩子数量>分箱完成,分箱结果为['无子女' '多胎' '两胎' '一胎' '非人类']


,年龄,收入,孩子数量,是否违约
0,46,0,无子女,0
1,34,3200,多胎,1
2,31,3300,多胎,1
3,39,1500,无子女,1
4,32,0,多胎,0


In [8]:
df22 = customize_bin(df=df11,variable="年龄",**{"dic1":{"未成年":"年龄 <=18","青年": "年龄 > 18 & 年龄< 35",
                                                  "中年":"年龄 >=35"}})
df22.head()

变量<年龄>分箱完成,分箱结果为['中年' '青年' '未成年']


,年龄,收入,孩子数量,是否违约
0,中年,0,无子女,0
1,青年,3200,多胎,1
2,青年,3300,多胎,1
3,中年,1500,无子女,1
4,青年,0,多胎,0
